## Imports

In [12]:
import os
import json
import joblib
from pathlib import Path
import shutil
from IPython.display import FileLink, display
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tensorflow import keras
from tensorflow.keras import layers
import librosa
import warnings
warnings.filterwarnings('ignore')

# setting random seed
tf.random.set_seed(42)
np.random.seed(42)

## Data Loading and Preprocessing

In [13]:
# loading YAMNet model from TensorFlow Hub
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')

# defining parameters
DATA_PATH = "/kaggle/input/dog-disease-sound-dataset"
SAMPLE_RATE = 16000
DURATION = 3.0

# defining class mapping
class_mapping = {
    'kennel_cough': 'Kennel Cough',
    'tracheal_collapse': 'Tracheal Collapse', 
    'reverse_sneeze': 'Reverse Sneeze',
    'normal_barking': 'Normal Barking',
    'normal_whining': 'Normal Whining',
    'normal_howling': 'Normal Howling',
    'normal_breathing': 'Normal Breathing',
    'normal_playing': 'Normal Playing'
}

In [14]:
def load_audio_files_with_yamnet_features(data_path, class_mapping):
    X = []  
    y = []  
    
    base_path = os.path.join(data_path, "expanded_dog_dataset")
    
    for class_dir, class_name in class_mapping.items():
        class_path = os.path.join(base_path, class_dir)
        
        if not os.path.exists(class_path):
            print(f"Warning: Directory {class_path} does not exist")
            continue
            
        print(f"Loading {class_name} audio files...")
        wav_files = [f for f in os.listdir(class_path) if f.endswith('.wav')]
        print(f"Found {len(wav_files)} .wav files in {class_dir}")
        
        for file in wav_files:
            file_path = os.path.join(class_path, file)
            
            try:
                # loading audio file
                audio, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
                
                # if audio is shorter than 3 seconds, pad it with zeros
                if len(audio) < SAMPLE_RATE * DURATION:
                    audio = np.pad(audio, (0, int(SAMPLE_RATE * DURATION) - len(audio)), mode='constant')
                
                # ensuring audio is in the correct range for YAMNet
                audio = audio.astype(np.float32)
                if np.max(np.abs(audio)) > 0:
                    audio = audio / np.max(np.abs(audio))
                
                # getting YAMNet embeddings
                scores, embeddings, spectrogram = yamnet_model(audio)
                
                # using the mean embedding as the feature vector
                mean_embedding = tf.reduce_mean(embeddings, axis=0).numpy()
                
                X.append(mean_embedding)
                y.append(class_name)
                
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
    
    return np.array(X), np.array(y)

# loading the data with YAMNet features
X, y = load_audio_files_with_yamnet_features(DATA_PATH, class_mapping)
print(f"Loaded {len(X)} audio samples with YAMNet features")
print(f"Feature vector shape: {X.shape}")

Loading Kennel Cough audio files...
Found 115 .wav files in kennel_cough
Loading Tracheal Collapse audio files...
Found 89 .wav files in tracheal_collapse
Loading Reverse Sneeze audio files...
Found 110 .wav files in reverse_sneeze
Loading Normal Barking audio files...
Found 157 .wav files in normal_barking
Loading Normal Whining audio files...
Found 150 .wav files in normal_whining
Loading Normal Howling audio files...
Found 151 .wav files in normal_howling
Loading Normal Breathing audio files...
Found 62 .wav files in normal_breathing
Loading Normal Playing audio files...
Found 91 .wav files in normal_playing
Loaded 925 audio samples with YAMNet features
Feature vector shape: (925, 1024)


## Label Encoding and Class Balancing

In [15]:
# encoding labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = tf.keras.utils.to_categorical(y_encoded)

# calculating class weights
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_encoded),
    y=y_encoded
)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)

# splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42, stratify=y_encoded
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_encoded[:len(X_train)]
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"Feature dimension: {X_train.shape[1]}")

Class weights: {0: 1.0054347826086956, 1: 0.7364649681528662, 2: 1.8649193548387097, 3: 0.765728476821192, 4: 1.2706043956043955, 5: 0.7708333333333334, 6: 1.0511363636363635, 7: 1.2991573033707866}
Training samples: 592
Validation samples: 148
Test samples: 185
Feature dimension: 1024


## Model Architecture

In [16]:
def create_yamnet_classifier(input_dim, num_classes):
    model = keras.Sequential([
        # building sequential neural network for classification
        layers.Dense(512, activation='relu', input_shape=(input_dim,)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# creating model
input_dim = X_train.shape[1]
num_classes = len(class_mapping)
model = create_yamnet_classifier(input_dim, num_classes)

# compiling model with Adam optimizer
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 693,640 (2.65 MB)

 Trainable params: 691,848 (2.64 MB)

 Non-trainable params: 1,792 (7.00 KB)

## Model Training and Evaluation

In [17]:
# defining callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    patience=15, 
    restore_best_weights=True,
    verbose=1,
    mode='max'
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-7,
    verbose=1,
    mode='max'
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'yamnet_based_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1,
    mode='max'
)

# training the model
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/100
16/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2734 - loss: 2.3772
Epoch 1: val_accuracy improved from -inf to 0.47297, saving model to yamnet_based_model.h5
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.2889 - loss: 2.3222 - val_accuracy: 0.4730 - val_loss: 1.6694 - learning_rate: 0.0010
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5793 - loss: 1.3446
Epoch 2: val_accuracy did not improve from 0.47297
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5780 - loss: 1.3474 - val_accuracy: 0.4595 - val_loss: 1.5632 - learning_rate: 0.0010
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5480 - loss: 1.1783
Epoch 3: val_accuracy improved from 0.47297 to 0.50676, saving model to yamnet_based_model.h5
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5478 - loss: 1.1803 - val_accuracy: 0.5068 - val_loss: 1.4933 - learning_rate: 0.0010
Epoch 4/100
16/19 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5995 - loss: 1.0463


In [18]:
# loading the best model
model = keras.models.load_model('yamnet_based_model.h5')

# evaluating on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, 
                           target_names=label_encoder.classes_))

# confusion matrix
import seaborn as sns
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix - YAMNet-based Model')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

Test Accuracy: 0.7189
Test Loss: 0.7963
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

Classification Report:
                   precision    recall  f1-score   support

     Kennel Cough       0.71      0.65      0.68        23
   Normal Barking       0.69      0.87      0.77        31
 Normal Breathing       0.92      0.85      0.88        13
   Normal Howling       0.78      0.83      0.81        30
   Normal Playing       0.60      0.33      0.43        18
   Normal Whining       0.75      0.70      0.72        30
   Reverse Sneeze       0.59      0.73      0.65        22
Tracheal Collapse       0.75      0.67      0.71        18

         accuracy                           0.72       185
        macro avg       0.72      0.70      0.71       185
     weighted avg       0.72      0.72      0.71       185



## Saving the Model

In [19]:
# creating directory for all model assets
os.makedirs('audio_model_assets', exist_ok=True)

# saving the trained model
model.save('audio_model_assets/dog_audio_model.h5')

# saving the label encoder
joblib.dump(label_encoder, 'audio_model_assets/label_encoder.pkl')

# saving class mapping
with open('audio_model_assets/class_mapping.json', 'w') as f:
    json.dump(class_mapping, f, indent=2)

# saving class names
class_names = label_encoder.classes_.tolist()
with open('audio_model_assets/class_names.json', 'w') as f:
    json.dump(class_names, f, indent=2)

# saving model configuration
model_config = {
    "model_type": "YAMNet-based Classifier",
    "input_dim": input_dim,
    "num_classes": num_classes,
    "sample_rate": SAMPLE_RATE,
    "duration": DURATION,
    "class_weights": class_weight_dict
}

with open('audio_model_assets/model_config.json', 'w') as f:
    json.dump(model_config, f, indent=2)

In [20]:
# creating inference script
inference_script = '''
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS
import joblib
import json
import tempfile
import os

class DogAudioClassifier:
    def __init__(self, model_assets_path):
        self.model_assets_path = model_assets_path
        
        # loading configuration
        with open(f'{model_assets_path}/model_config.json', 'r') as f:
            self.config = json.load(f)
        
        # loading label encoder
        self.label_encoder = joblib.load(f'{model_assets_path}/label_encoder.pkl')
        
        # loading class names
        with open(f'{model_assets_path}/class_names.json', 'r') as f:
            self.class_names = json.load(f)
        
        # loading model
        self.model = tf.keras.models.load_model(f'{model_assets_path}/dog_audio_model.h5')
        
        self.SAMPLE_RATE = self.config["sample_rate"]
        self.DURATION = self.config["duration"]
        
        # loading YAMNet model
        self.yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
    
    def extract_yamnet_features(self, audio_path):
        # extracting YAMNet features from audio file
        try:
            # loading and preprocessing audio
            audio, sr = librosa.load(audio_path, sr=self.SAMPLE_RATE, duration=self.DURATION)
            
            # padding if shorter than duration
            if len(audio) < self.SAMPLE_RATE * self.DURATION:
                audio = np.pad(audio, (0, int(self.SAMPLE_RATE * self.DURATION) - len(audio)), mode='constant')
            
            # normalizing audio
            audio = audio.astype(np.float32)
            if np.max(np.abs(audio)) > 0:
                audio = audio / np.max(np.abs(audio))
            
            # getting YAMNet embeddings
            scores, embeddings, spectrogram = self.yamnet_model(audio)
            mean_embedding = tf.reduce_mean(embeddings, axis=0).numpy()
            
            return mean_embedding
            
        except Exception as e:
            raise Exception(f"Audio processing error: {str(e)}")
    
    def predict(self, audio_path, top_k=3):
        try:
            # extracting features
            features = self.extract_yamnet_features(audio_path)
            features = features.reshape(1, -1)
            
            # getting prediction
            predictions = self.model.predict(features, verbose=0)
            top_k = min(top_k, self.config["num_classes"])
            
            # getting top predictions
            top_indices = np.argsort(predictions[0])[-top_k:][::-1]
            top_probs = predictions[0][top_indices]
            
            results = []
            for i, (idx, prob) in enumerate(zip(top_indices, top_probs)):
                disease = self.label_encoder.inverse_transform([idx])[0]
                
                # generating confidence explanation
                if prob > 0.7:
                    confidence_level = "High confidence"
                    explanation = "Clear audio patterns match this condition"
                elif prob > 0.5:
                    confidence_level = "Moderate confidence" 
                    explanation = "Good audio alignment with some uncertainty"
                elif prob > 0.3:
                    confidence_level = "Low confidence"
                    explanation = "Audio features could indicate multiple conditions"
                else:
                    confidence_level = "Very low confidence"
                    explanation = "Limited audio information available"
                
                results.append({
                    'disease': disease, 
                    'confidence': float(prob),
                    'confidence_level': confidence_level,
                    'explanation': explanation,
                    'class_index': int(idx)
                })
            
            return {
                'predictions': results,
                'top_disease': results[0]['disease'],
                'top_confidence': results[0]['confidence'],
                'status': 'success'
            }
            
        except Exception as e:
            return {
                'error': str(e),
                'status': 'error'
            }

# flask app
app = Flask(__name__)
CORS(app)

# initializing predictor
predictor = None

def initialize_predictor():
    global predictor
    if predictor is None:
        predictor = DogAudioClassifier('audio_model_assets')
    return predictor

@app.route('/predict-audio', methods=['POST'])
def predict_audio_endpoint():
    if 'audio' not in request.files:
        return jsonify({'error': 'No audio file provided', 'status': 'error'}), 400
    
    file = request.files['audio']
    if file.filename == '':
        return jsonify({'error': 'No file selected', 'status': 'error'}), 400
    
    try:
        predictor = initialize_predictor()
        
        # saving uploaded file temporarily
        with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as tmp_file:
            file.save(tmp_file.name)
            result = predictor.predict(tmp_file.name)
        
        os.unlink(tmp_file.name)
        
        return jsonify(result)
    
    except Exception as e:
        return jsonify({'error': str(e), 'status': 'error'}), 500

@app.route('/health-audio', methods=['GET'])
def health_check():
    predictor = initialize_predictor()
    return jsonify({
        'status': 'healthy', 
        'model_loaded': True,
        'num_classes': len(predictor.class_names),
        'classes': predictor.class_names
    })

if __name__ == '__main__':
    initialize_predictor()
    print("Dog Audio Disease Prediction API started!")
    print(f"Loaded {len(predictor.class_names)} classes: {predictor.class_names}")
    app.run(host='0.0.0.0', port=5001, debug=True)
'''

with open('audio_model_assets/inference.py', 'w') as f:
    f.write(inference_script)

In [21]:
# creating requirements.txt
requirements = '''
tensorflow>=2.8.0
tensorflow_hub>=0.12.0
librosa>=0.9.0
scikit-learn>=1.0.0
numpy>=1.21.0
flask>=2.0.0
flask-cors>=3.0.0
joblib>=1.0.0
'''

with open('audio_requirements.txt', 'w') as f:
    f.write(requirements)

shutil.copy('audio_requirements.txt', 'audio_model_assets/requirements.txt')

'audio_model_assets/requirements.txt'

In [22]:
# creating zip archive of model assets
shutil.make_archive('/kaggle/working/audio_model_assets', 'zip', '/kaggle/working/audio_model_assets')

# creating zip of essential files
essential_files = [
    'audio_requirements.txt',
    'audio_model_assets/inference.py',
    'audio_model_assets/dog_audio_model.h5',
    'audio_model_assets/label_encoder.pkl',
    'audio_model_assets/class_names.json',
    'audio_model_assets/model_config.json'
]

os.makedirs('/kaggle/working/audio_essential_files', exist_ok=True)
for file_path in essential_files:
    if os.path.exists(file_path):
        file_name = os.path.basename(file_path)
        shutil.copy(file_path, f'/kaggle/working/audio_essential_files/{file_name}')

shutil.make_archive('/kaggle/working/audio_essential_files', 'zip', '/kaggle/working/audio_essential_files')

'/kaggle/working/audio_essential_files.zip'